<a href="https://colab.research.google.com/github/lautitorraca/7506R-2C2023-GRUPO24/blob/main/TP2/7506R_TP2_GRUPO24_RED_NEURONAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

𝔾𝕣𝕦𝕡𝕠 𝟚𝟜 (Merequetengue👍)

➢ Torraca Lautaro - 108813

➢ Negrotti Gianluca - 108184

➢ Marco Tosi - 107237

### > Imports

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras.metrics import BinaryAccuracy
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow import keras
from sklearn.preprocessing import StandardScaler

# Optimización de hiperparámetros
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RepeatedKFold

# Manejo de modelos
from pickle import dump
from pickle import load

### > Lectura de los datasets

In [ ]:
url_train = '/content/drive/MyDrive/Colab Notebooks/train_limpio.csv'
url_test = '/content/drive/MyDrive/Colab Notebooks/test_limpio.csv'


df_train = pd.read_csv(url_train)
df_test = pd.read_csv(url_test)

train = df_train.copy()
test = df_test.copy()

- Division y preprocesamiento de los datos



In [ ]:
target = 'sentimiento'
features = 'review_es_clean'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train[features],
                                                    train[target],
                                                    test_size=0.05,
                                                    random_state=42,
                                                    stratify=train[target],
                                                    shuffle=True)



### Red neuronal

- Tokenizacion



In [ ]:
# Convertir las críticas de texto a secuencias numéricas
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index

# Ajustar el tamaño del vocabulario en la capa Embedding
vocab_size = min(len(word_index) + 1, 10000)

# Limitar la longitud de las secuencias a un tamaño fijo
max_sequence_length = 128
X_train_padded = pad_sequences(sequences, maxlen=max_sequence_length)

# Convertir las etiquetas a valores numéricos
labels = np.array(y_train.replace({"negativo": 0, "positivo": 1}))


- Creacion de funcion para optimizar f1

In [ ]:
def f1_score(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)


- Creacion del modelo de RNN

In [ ]:
# Crear el modelo de RNN con capas GRU
model_opt = Sequential()
model_opt.add(Embedding(vocab_size, 100, input_length=max_sequence_length))
model_opt.add(GRU(64, dropout=0.2))
model_opt.add(Dense(1, activation='sigmoid'))

# Configurar el optimizador
optimizer = optimizers.Adam()

# Compilar el modelo
model_opt.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[BinaryAccuracy(name='accuracy')])




- Entrenamiento del modelo

In [ ]:
# Compilar el modelo
model_opt.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=[f1_score])

# Cantidad de épocas de entrenamiento
cant_epochs = 8

# Configurar el callback de Early Stopping para monitorear el f1_score
early_stopping = EarlyStopping(monitor='val_f1_score',
                               patience=4,
                               mode='max',
                               restore_best_weights=True,
                               min_delta=0.0001)

# Entrenar el modelo
history_model_rn = model_opt.fit(X_train_padded,
                                 labels,
                                 epochs=cant_epochs,
                                 batch_size=32,
                                 validation_split=0.2,
                                 verbose=1,
                                 callbacks=[early_stopping])


Epoch 1/8
1188/1188 [==============================] - 146s 120ms/step - loss: 0.4073 - f1_score: 0.8115 - val_loss: 0.3402 - val_f1_score: 0.8457
Epoch 2/8
1188/1188 [==============================] - 141s 118ms/step - loss: 0.2614 - f1_score: 0.8927 - val_loss: 0.3328 - val_f1_score: 0.8564
Epoch 3/8
1188/1188 [==============================] - 145s 122ms/step - loss: 0.1901 - f1_score: 0.9263 - val_loss: 0.3814 - val_f1_score: 0.8622
Epoch 4/8
1188/1188 [==============================] - 140s 118ms/step - loss: 0.1318 - f1_score: 0.9499 - val_loss: 0.4441 - val_f1_score: 0.8481
Epoch 5/8
1188/1188 [==============================] - 143s 120ms/step - loss: 0.0887 - f1_score: 0.9679 - val_loss: 0.5029 - val_f1_score: 0.8358
Epoch 6/8
1188/1188 [==============================] - 142s 120ms/step - loss: 0.0625 - f1_score: 0.9769 - val_loss: 0.6554 - val_f1_score: 0.8349
Epoch 7/8
1188/1188 [==============================] - 140s 118ms/step - loss: 0.0466 - f1_score: 0.9831 - val_loss: 0

- Prediccion y muestra de la matriz

In [ ]:
from sklearn.metrics import confusion_matrix
import numpy as np


# Convertir las críticas de texto de prueba a secuencias numéricas
sequences_test = tokenizer.texts_to_sequences(X_test)
X_test_padded = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Realizar predicciones
predictions = model_opt.predict(X_test_padded)

# Convertir las predicciones a etiquetas binarias
predicted_labels = [1 if p >= 0.5 else 0 for p in np.ravel(predictions)]

# Convertir y_test a valores numéricos si es necesario
y_test_numeric = np.array(y_test.replace({"negativo": 0, "positivo": 1}))

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test_numeric, predicted_labels)
print("Matriz de Confusión:")
print(conf_matrix)


79/79 [==============================] - 3s 27ms/step
Matriz de Confusión:
[[1029  221]
 [ 120 1130]]


- Guardado del modelo en H5 (similar a pickle)

In [ ]:
model_opt.save('red_neuronal.h5')  # Guarda el modelo en formato H5
from google.colab import files
files.download('red_neuronal.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- Prediccion con el dataset Test

In [ ]:
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Convertir las revisiones de texto a secuencias numéricas
test_sequences = tokenizer.texts_to_sequences(test['review_es_clean'])


max_sequence_length = 200
X_test_tokenizer = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Realizar predicciones
predictions = model_opt.predict(X_test_tokenizer)
predicted_sentiment = ['positivo' if pred >= 0.5 else 'negativo' for pred in predictions]

# Agregar las predicciones al DataFrame de prueba
test['sentimiento'] = predicted_sentiment

# Seleccionar solo las columnas ID y predicted_sentiment
output = test[['ID', 'sentimiento']]

# Guardar las predicciones en un nuevo archivo CSV
output.to_csv('prediccion_RN.csv', index=False)


269/269 [==============================] - 9s 31ms/step


- Descarga de la prediccion

In [ ]:
# Descargar el archivo (útil en Jupyter Notebook o Google Colab)
try:
    from google.colab import files
    files.download('prediccion_RN.csv')
except ImportError:
    print("La función de descarga solo funciona en entornos como Google Colab.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### >  Segundo modelo de Red Neuronal

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences




# Codificación de las etiquetas
label_encoder = LabelEncoder()
train['sentimiento_encoded'] = label_encoder.fit_transform(train['sentimiento'])

# Vectorización de las reseñas
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train['review_es_clean'])

X = tokenizer.texts_to_sequences(train['review_es_clean'])
X = pad_sequences(X, maxlen=200) # Ajusta este valor según la longitud de tus reseñas

y = train['sentimiento_encoded']

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Parámetros del modelo
cant_clases = len(np.unique(y_train))
d_in = X_train.shape[1]

modelo_1 = keras.Sequential([
    keras.layers.Dense(32, input_shape=(d_in,), activation='relu', kernel_initializer='uniform'), # Capa de entrada
    keras.layers.Dense(32, activation='relu'),

    keras.layers.Dropout(0.6),

    keras.layers.Dense(32, input_shape=(d_in,), activation='relu', kernel_initializer='uniform'),
    keras.layers.Dense(32, activation='relu'),

    keras.layers.Dropout(0.6),
    keras.layers.Dense(cant_clases, activation='sigmoid') # Capa de salida
])

modelo_1.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])


In [ ]:
history = modelo_1.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
1000/1000 [==============================] - 5s 3ms/step - loss: 0.9735 - accuracy: 0.4987 - val_loss: 0.6930 - val_accuracy: 0.5041
Epoch 2/5
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6939 - accuracy: 0.4978 - val_loss: 0.6938 - val_accuracy: 0.5004
Epoch 3/5
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6941 - accuracy: 0.5001 - val_loss: 0.6932 - val_accuracy: 0.5040
Epoch 4/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6940 - accuracy: 0.4995 - val_loss: 0.6932 - val_accuracy: 0.5026
Epoch 5/5
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6936 - accuracy: 0.4965 - val_loss: 0.6931 - val_accuracy: 0.5081


- Evaluacion y prediccion del modelo

In [ ]:
# Evaluación del modelo
test_loss, test_acc = modelo_1.evaluate(X_test, y_test)

print('Test Accuracy:', test_acc)

# Realizar predicciones
predictions = modelo_1.predict(X_test)


313/313 [==============================] - 1s 2ms/step - loss: 0.6931 - accuracy: 0.4988
Test Accuracy: 0.49880000948905945
313/313 [==============================] - 1s 2ms/step


- Guardado del archivo con pickle

In [ ]:
import pickle

# Guardar el modelo
with open('modelo_1.pkl', 'wb') as file:
    pickle.dump(modelo_1, file)


- Prediccion con el dataset de test

In [ ]:

X_nuevo = tokenizer.texts_to_sequences(test['review_es_clean'])
X_nuevo = pad_sequences(X_nuevo, maxlen=200)  # Usa la misma longitud máxima que antes


In [ ]:
nuevas_predicciones = modelo_1.predict(X_nuevo)


269/269 [==============================] - 0s 2ms/step


- Preparacion del csv para bajar

In [ ]:


# Convertir las salidas del modelo en índices de clase
indices_predicciones = np.argmax(nuevas_predicciones, axis=1)


mapeo_etiquetas = {0: 'Negativo', 1: 'Positivo'}
etiquetas_predicciones = [mapeo_etiquetas[i] for i in indices_predicciones]


- Bajado del archivo de prediccion

In [ ]:
# Supongamos que 'nuevo_dataset' tiene una columna 'ID'
nuevo_dataset_predicciones = pd.DataFrame({
    'ID': test['ID'],
    'sentimiento': etiquetas_predicciones
})

# Guardar en un archivo CSV
nuevo_dataset_predicciones.to_csv('predicciones_red2.csv', index=False)
try:
    from google.colab import files
    files.download('predicciones_red2.csv')
except ImportError:
    print("La función de descarga solo funciona en entornos como Google Colab.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>